In [8]:
import pandas as pd
from signal_metadata_extraction import MetadataExtractor

In [ ]:
files = ['train_data.csv', 'test_data.csv', 'valid_data.csv']

for file in files:

    metadata_extractor = MetadataExtractor()
    metadatas = []
    audio_col = 'path'

    df_divExplorer = pd.read_csv(file)

    from tqdm import tqdm
    for i in tqdm(range(len(df_divExplorer))):

        audio_file = df_divExplorer[audio_col].iloc[i]
        signal_metadata = metadata_extractor.signal_metadata(audio_file)
        
        sentence = df_divExplorer['transcription'].iloc[i] 
        text_metadata = metadata_extractor.text_metadata(sentence)
        mixed_metadata = metadata_extractor.mixed_metadata(signal_metadata, text_metadata)
        metadatas.append(signal_metadata + text_metadata + mixed_metadata)

    meta_cols = metadata_extractor.list_signal_metadata \
            + metadata_extractor.list_text_metadata \
            + metadata_extractor.list_sig_text_metadata
    df_divExplorer_metas = pd.concat([df_divExplorer, pd.DataFrame(metadatas, columns=meta_cols)], axis=1)

    print("Metadata columns: ", meta_cols)
    print("---")
    print("Df for DivExplorer Columns: ", df_divExplorer_metas.columns)

    ## Merge df_divExplorer_metas with df_demo on 'speakerId'
    df_divExplorer_metas_demo = pd.merge(df_divExplorer_metas, df_demo, on='speakerId', how='left')
    print(len(df_divExplorer_metas_demo), len(df_divExplorer_metas), len(df_demo))

    df_divExplorer_metas_demo.to_csv(file, index=False)